In [1]:
import warnings
warnings.filterwarnings('ignore')
import os
import re
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, accuracy_score
from sklearn.metrics import roc_auc_score, roc_curve, confusion_matrix
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.datasets import fetch_20newsgroups, load_files

import pandas as pd
from scipy.sparse import csr_matrix
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

df = pd.read_csv('../Desktop/Dataset/bank_train.csv')
labels = pd.read_csv('../Desktop/Dataset//bank_train_target.csv', header=None)

df.head()

age          job  marital          education  default housing loan  \
0   26      student   single        high.school       no      no   no   
1   46       admin.  married  university.degree       no     yes   no   
2   49  blue-collar  married           basic.4y  unknown     yes  yes   
3   31   technician  married  university.degree       no      no   no   
4   42    housemaid  married  university.degree       no     yes   no   

     contact month day_of_week  duration  campaign  pdays  previous  \
0  telephone   jun         mon       901         1    999         0   
1   cellular   aug         tue       208         2    999         0   
2  telephone   jun         tue       131         5    999         0   
3   cellular   jul         tue       404         1    999         0   
4  telephone   nov         mon        85         1    999         0   

      poutcome  emp.var.rate  cons.price.idx  cons.conf.idx  euribor3m  \
0  nonexistent           1.4          94.465          -41.8      4.961   
1  nonexistent           1.4          93.444          -36.1      4.963   
2  nonexistent           1.4          94.465          -41.8      4.864   
3  nonexistent          -2.9          92.469          -33.6      1.044   
4  nonexistent          -0.1          93.200          -42.0      4.191   

   nr.employed  
0       5228.1  
1       5228.1  
2       5228.1  
3       5076.2  
4       5195.8

In [12]:
for s in ('university.degree', 'high.school', 'illiterate'):
    print(s, '->', hash(s))

university.degree -> 4696435823864105068
high.school -> -9185765955227575813
illiterate -> -7530998789830211300


Отрицательные и настолько большие по модулю значения нам не подойдут. Ограничим область значений хэш-функции:



In [13]:
hash_space = 25
for s in ('university.degree', 'high.school', 'illiterate'):
    print(s, '->', hash(s) % hash_space)

university.degree -> 18
high.school -> 12
illiterate -> 0


In [7]:
hashing_example = pd.DataFrame([{i: 0.0 for i in range(hash_space)}])
for s in ('job=student', 'marital=single', 'day_of_week=mon'):
    print(s, '->', hash(s) % hash_space)
    hashing_example.loc[0, hash(s) % hash_space] = 1
hashing_example

job=student -> 7
marital=single -> 9
day_of_week=mon -> 0


0    1    2    3    4    5    6    7    8    9   ...   15   16   17   18  \
0  1.0  0.0  0.0  0.0  0.0  0.0  0.0  1.0  0.0  1.0  ...  0.0  0.0  0.0  0.0   

    19   20   21   22   23   24  
0  0.0  0.0  0.0  0.0  0.0  0.0  

[1 rows x 25 columns]

In [14]:
assert hash('no') == hash('no')
assert hash('housing=no') != hash('loan=no')